In [8]:
import torch
torch.cuda.empty_cache()

In [9]:
import torch
from transformers import LlamaForCausalLM, PreTrainedTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset
import transformers
from datasets import Dataset
import pandas as pd
import os

In [18]:
# LLaMA 3 한국어 모델 로드
model_name = "psymon/KoLlama2-7b"
model = LlamaForCausalLM.from_pretrained(model_name, device_map="auto", low_cpu_mem_usage=True)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, legacy=False)  # Fast Tokenizer 사용 및 legacy 옵션 비활성화

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

C:\Users\문영주\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\문영주\.cache\huggingface\hub\models--psymon--KoLlama2-7b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.

In [19]:

# 데이터셋 파일 경로
data_path = r'C:\Users\문영주\Desktop\Python_projects\LLM\doctorbot'
file_names = [f"dataset_v{i}.csv" for i in range(1, 9)]

# 데이터 병합
data_frames = [pd.read_csv(os.path.join(data_path, file)) for file in file_names]
combined_data = pd.concat(data_frames, ignore_index=True)

# 중복 제거
combined_data.drop_duplicates(inplace=True)


In [20]:
combined_data = combined_data[['질문', '소견']]

In [21]:
# Hugging Face Dataset 포맷으로 변환
dataset = Dataset.from_pandas(combined_data)

In [22]:
dataset

Dataset({
    features: ['질문', '소견', '__index_level_0__'],
    num_rows: 2314
})

In [23]:
# 데이터셋 전처리 함수
def preprocess(example):
    # '질문'과 '소견' 컬럼을 사용하여 입력 텍스트 생성
    input_text = "질문: " + example['질문'] + " 소견: " + example['소견']
    # 텍스트를 tokenization하고 'input_ids' 컬럼에 추가
    example['input_ids'] = tokenizer(input_text, return_tensors="pt").input_ids.squeeze()
    return example

# 데이터셋 전처리
dataset = dataset.map(preprocess, remove_columns=['질문', '소견'])


training_args = TrainingArguments(
    output_dir="./llama3-sft",
    eval_strategy="epoch",  # evaluation_strategy -> eval_strategy로 수정
    #evaluation_strategy="epoch",  # 이 줄을 아래와 같이 수정해야 합니다.
    logging_dir="./logs",
    # 그 외의 설정...
)



Map:   0%|          | 0/2314 [00:00<?, ? examples/s]

In [16]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: B883-EF89

 C:\Users\문영주\Desktop\Python_projects\LLM 디렉터리

2024-11-07  오후 09:32    <DIR>          .
2024-09-24  오후 08:14    <DIR>          ..
2024-11-07  오후 08:56    <DIR>          .ipynb_checkpoints
2024-04-12  오후 09:06    <DIR>          chemical
2024-11-07  오후 09:14    <DIR>          doctorbot
2024-02-03  오후 08:11    <DIR>          fast campus
2024-02-19  오후 06:23    <DIR>          koinsurance_LLM
2024-11-07  오후 09:32            11,015 Untitled.ipynb
2024-05-30  오후 10:04    <DIR>          상품약관
               1개 파일              11,015 바이트
               8개 디렉터리  486,699,061,248 바이트 남음


In [24]:

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset

)

# 모델 학습
trainer.train()

RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

In [ ]:
http://localhost:8888/lab/tree/Desktop/Python_projects/LLM/~/AppData/Roaming/Python/Python311/site-packages/transformers/trainer.py#line=527